In [1]:
import pandas as pd
import glob
import warnings
from pprint import pprint
from pathlib import Path
from tqdm.notebook import tqdm

import nltk

# Imports

In [2]:
DATA_DIR = Path("../../data")
MODEL_DIR = Path("../../model")
input_dir = DATA_DIR / "processed" / "nyt-data"

In [3]:
file_paths = glob.glob(str(input_dir / "*.csv"))

In [4]:
file_paths

['../../data/processed/nyt-data/2000_data.csv',
 '../../data/processed/nyt-data/2001_data.csv',
 '../../data/processed/nyt-data/1997_data.csv',
 '../../data/processed/nyt-data/1996_data.csv',
 '../../data/processed/nyt-data/2016_data.csv',
 '../../data/processed/nyt-data/1990_data.csv',
 '../../data/processed/nyt-data/1991_data.csv',
 '../../data/processed/nyt-data/2010_data.csv',
 '../../data/processed/nyt-data/2011_data.csv',
 '../../data/processed/nyt-data/2007_data.csv',
 '../../data/processed/nyt-data/2006_data.csv',
 '../../data/processed/nyt-data/1994_data.csv',
 '../../data/processed/nyt-data/1995_data.csv',
 '../../data/processed/nyt-data/2014_data.csv',
 '../../data/processed/nyt-data/2015_data.csv',
 '../../data/processed/nyt-data/2009_data.csv',
 '../../data/processed/nyt-data/2008_data.csv',
 '../../data/processed/nyt-data/2003_data.csv',
 '../../data/processed/nyt-data/2002_data.csv',
 '../../data/processed/nyt-data/2004_data.csv',
 '../../data/processed/nyt-data/2005_dat

In [5]:
dataframes = []
for file in file_paths:
    df = pd.read_csv(file)
    dataframes.append(df)

In [6]:
# used for training the Cade-Compass
df_all = pd.concat(dataframes, ignore_index=True)

In [7]:
df_all.head()

,id,date,text,year
0,4fd232cd8eb7c8105d7c2d56,2000-01-01,island latest pristine cuisine restaurant fash...,2000
1,4fd210ee8eb7c8105d785b4c,2000-01-01,moscow dec 31 newborn political party register...,2000
2,4fd21fb88eb7c8105d7a2ee3,2000-01-01,washington dec 31 vice president gore denounce...,2000
3,4fd233c08eb7c8105d7c43b7,2000-01-01,harris doesnt killers face saw walking street ...,2000
4,4fd233c08eb7c8105d7c43af,2000-01-01,tax court judge ruled one nations prominent ta...,2000


In [8]:
print(f"Imported {len(dataframes)} files")

for i, df in enumerate(dataframes):
    print(f"Data from Period {i+1}: {df.date.min()} - {df.date.max()}")

Imported 27 files
Data from Period 1: 2000-01-01 - 2000-12-31
Data from Period 2: 2001-01-01 - 2001-12-31
Data from Period 3: 1997-01-01 - 1997-09-10
Data from Period 4: 1996-01-01 - 1996-12-31
Data from Period 5: 2016-01-01 - 2016-06-05
Data from Period 6: 1990-01-01 - 1990-10-31
Data from Period 7: 1991-01-01 - 1991-12-31
Data from Period 8: 2010-01-01 - 2010-12-23
Data from Period 9: 2011-01-01 - 2011-12-15
Data from Period 10: 2007-01-01 - 2007-12-20
Data from Period 11: 2006-01-01 - 2006-12-22
Data from Period 12: 1994-01-01 - 1994-12-31
Data from Period 13: 1995-01-01 - 1995-12-31
Data from Period 14: 2014-01-01 - 2014-12-31
Data from Period 15: 2015-01-01 - 2015-12-31
Data from Period 16: 2009-01-01 - 2009-12-20
Data from Period 17: 2008-01-01 - 2008-12-21
Data from Period 18: 2003-01-01 - 2003-12-31
Data from Period 19: 2002-01-01 - 2002-12-31
Data from Period 20: 2004-01-01 - 2004-12-31
Data from Period 21: 2005-01-01 - 2005-12-29
Data from Period 22: 1999-01-01 - 1999-12-31
D

In [9]:
# convert date back to datetime object and sort them by date
for df in dataframes:
    df['date'] =  pd.to_datetime(df['date'])
    df.sort_values('date', inplace=True)

df_all['date'] =  pd.to_datetime(df_all['date'])
df_all.sort_values('date', inplace=True)

Check if any missing values are in the (processed) text column

In [10]:
df_all["text"].isna().sum()

0

In [11]:
df_all = df_all.dropna(subset=["text"])

# Create Corpora

In [12]:
corpus_all = df_all["text"].values.tolist()

corpora = [df["text"].values.tolist() for df in dataframes]

In [13]:
len(corpus_all)

98360

In [14]:
len(corpora)

27

In [15]:
pprint(corpus_all[0:500])

['third national basketball association season completed knicks 207 record '
 'tops eastern conference league los angeles lakers 216 knicks still best '
 'conference record next 15 games jackson coach east team 40th nba allstar '
 'game feb 11 miami coaches teams best conference records games jan 28 coach '
 'allstar teams saturdays games knicks best record east game half ahead '
 'chicago bulls indiana pacers bulls pacers tied 199 central division lead '
 'detroit pistons atlanta hawks counted jackson bulls phil jackson firstyear '
 'coaches three rookie coaches honor coaching allstar game easy ed st louis '
 'hawks accomplished feat 1959 billy cunningham philadelphia 76ers 1978 pat '
 'riley lakers 1982 knicks streak unlike jacksons none three started season '
 'teams instead replacements dismissed coaches took andy phillips 10 games '
 'cunningham replaced gene 6 games riley succeeded paul 11 games knicks '
 'extended winning streak nine games saturday beating orlando magic ran '
 '

# Create Tokens

In [16]:
tokenizer = nltk.tokenize.TreebankWordTokenizer()

tokens_all = tokenizer.tokenize(" ".join(str(text) for text in tqdm(corpus_all)))

tokens = [tokenizer.tokenize(" ".join(str(text) for text in corpus)) for corpus in tqdm(corpora)]

  0%|          | 0/98360 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [17]:
len(tokens_all)

49721718

# Create Vocabularies

In [18]:
vocabulary_all = set(tokens_all)

vocabularies = [set(tokens_split) for tokens_split in tqdm(tokens)]

  0%|          | 0/27 [00:00<?, ?it/s]

In [19]:
vocabulary_size_all = len(vocabulary_all)

vocabulary_sizes = [len(vocabulary) for vocabulary in vocabularies]

In [20]:
print(f"Vocabulary size: {vocabulary_size_all}")
for i, vocab_size in enumerate(vocabulary_sizes):
    print(f"Vocabulary size of split {i+1}: {vocab_size}")

Vocabulary size: 22062
Vocabulary size of split 1: 21756
Vocabulary size of split 2: 21780
Vocabulary size of split 3: 21357
Vocabulary size of split 4: 21602
Vocabulary size of split 5: 21334
Vocabulary size of split 6: 21238
Vocabulary size of split 7: 21387
Vocabulary size of split 8: 21798
Vocabulary size of split 9: 21820
Vocabulary size of split 10: 21931
Vocabulary size of split 11: 19104
Vocabulary size of split 12: 21503
Vocabulary size of split 13: 21520
Vocabulary size of split 14: 21856
Vocabulary size of split 15: 21853
Vocabulary size of split 16: 21810
Vocabulary size of split 17: 21935
Vocabulary size of split 18: 21836
Vocabulary size of split 19: 21822
Vocabulary size of split 20: 21906
Vocabulary size of split 21: 21906
Vocabulary size of split 22: 21709
Vocabulary size of split 23: 21699
Vocabulary size of split 24: 21439
Vocabulary size of split 25: 21437
Vocabulary size of split 26: 21870
Vocabulary size of split 27: 21902


# Word Embeddings

1. Static: Word2Vec (Gensim)

2. Temporal: TWEC/CADE

## 1. Word2Vec (static)

In [21]:
from gensim.models import Word2Vec

In [22]:
from nltk import word_tokenize

tokenized_sentences_all = [word_tokenize(item) for item in tqdm(corpus_all)]

  0%|          | 0/98360 [00:00<?, ?it/s]

In [23]:
w2v_model = Word2Vec(tokenized_sentences_all, seed=1040)

In [24]:
test_word = "apple"
w2v_model.wv.most_similar(test_word)

[('apples', 0.8348197937011719),
 ('samsung', 0.7517530918121338),
 ('dell', 0.6891453862190247),
 ('android', 0.6865653991699219),
 ('macintosh', 0.6797322034835815),
 ('blackberry', 0.6770710349082947),
 ('intel', 0.6706156134605408),
 ('iphone', 0.6664397716522217),
 ('pc', 0.6633753776550293),
 ('ibm', 0.656183660030365)]

In [25]:
output_model_dir_static = MODEL_DIR / "nyt-data" / "static"

output_model_dir_static.mkdir(parents=True, exist_ok=True)

model_static_file_path = output_model_dir_static / "w2v_model.model"

w2v_model.save(str(model_static_file_path))

## 2. Temporal: TWEC/CADE

***Temporal Word Embeddings with a Compass***

* [Source-Code](https://github.com/valedica/twec)

* [Paper](https://arxiv.org/abs/1906.02376)

* [Blogpost](https://fede-bianchi.medium.com/aligning-temporal-diachronic-word-embeddings-with-a-compass-732ab7427955)

Save the concatenated text to txt-files to make them usable for Cade

In [26]:
import smart_open
print(smart_open.__version__)

1.10.0


In [27]:
from cade.cade import CADE

In [28]:
# Declare Paths
split_dir = DATA_DIR / "processed" / "nyt-data"
cade_dir = MODEL_DIR / "nyt-data" / "cade" / "data"

cade_dir.mkdir(parents=True, exist_ok=True)

# Create filepaths/names
file_paths = [(cade_dir / csv_file.split("/")[-1].split(".")[0]).with_suffix(".txt") for csv_file in file_paths]

file_paths_and_corpora = {
    cade_dir / 'compass.txt': corpus_all
}

for key, value in zip(file_paths, corpora):
    file_paths_and_corpora[key] = value

In [29]:
# Write them to memory
for file_path, corpus in file_paths_and_corpora.items():
    with open(file_path, 'w') as file:
        for item in corpus:
            file.write("%s\n" % item)

Create & train the compass

This creates atemporal context and target word embeddings

In [30]:
import gensim
print(gensim.__version__)

3.8.0


In [31]:
warnings.filterwarnings("ignore")

path_to_cade_models = Path("../../model/nyt-data/cade/model")
path_to_cade_models.mkdir(parents=True, exist_ok=True)

aligner = CADE(size=30, min_count = 5, opath=path_to_cade_models)
aligner.train_compass(str((cade_dir / "compass").with_suffix(".txt")), overwrite=True);

Training the compass from scratch.


In [33]:
warnings.filterwarnings("ignore")

# train slices, they will be already aligned
slices = [aligner.train_slice(file_path, save=True) for file_path in file_paths] # list of gensim word2vec objects

Training embeddings: slice ../../model/nyt-data/cade/data/2000_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/2001_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/1997_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/1996_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/2016_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/1990_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/1991_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/2010_data.txt.
Initializing embeddings from compass.
Training embeddings: slice ../../model/nyt-data/cade/data/2011_data.txt.
Initializing embeddings from compass.
T

In [ ]:
print("done")